# Demonstration using NEURON + Python

This jupyter notebook requires a valid installation of [NEURON](https://github.com/nrnhines/nrn) with python configured. 

### 1. Download a neuron morphology and electrical model from ModelDB


This model is open source and provided by the Allen Brain Institute

In [1]:
import http
import requests
import shutil

def modeldb_download(model_id, fileobj):
    """Download model from ModelDB in ZIP format
    """
    dl_url_prefix = "https://senselab.med.yale.edu/modeldb/eavBinDown.cshtml?"
    dl_url = dl_url_prefix + "o={model_id}&a=311&mime=application/zip"
    resp = requests.get(dl_url.format(model_id=model_id), stream=True)
    resp.raise_for_status()
    if resp.status_code == httplib.OK:
        resp.raw.decode_content = True
        shutil.copyfileobj(resp.raw, fileobj)

ImportError: No module named 'requests'

In [ ]:
# Download Allen Cell Types Database model in ./models directory
import os
try:
    os.makedirs('/opt/src/models')
except OSError:
    pass
with open('/opt/src/models/184167.zip', 'wb') as ostr:
    modeldb_download(184167, ostr)

In [ ]:
! cd /opt/src/models ; unzip 184167

In [ ]:
! rm -rf /opt/src/models/AllenBrainNeuron ; mv /opt/src/models/472300877converted /opt/src/models/AllenBrainNeuron


### 2. Download our special synapse model

This model is very likely open source. Anyways, use with care. 

#### To download the model, replace `<user>` below with your username 

In [ ]:
! git clone -q \
  ssh://$USER_LOGIN@bbpcode.epfl.ch/sim/neurodamus/bbp /opt/src/neurodamus

In [ ]:
! cp /opt/src/neurodamus/lib/modlib/ProbAMPANMDA_EMS.mod /opt/src/models/AllenBrainNeuron/modfiles

In [ ]:
os.chdir('/opt/src/models/AllenBrainNeuron')

### 3. Generate ion channel and synapse C files for NEURON

In [ ]:
! nrnivmodl modfiles

### 4. Definition of simulation parameters

In [ ]:
import neuron
from neuron import h,gui
import matplotlib.pyplot as plt
import sys
import numpy as np

%matplotlib inline


In [ ]:
sys.path.append(os.getcwd())
from neuron472300877 import Neuron472300877 as Cell

In [ ]:
junction_potential = -14.0
h.tstop = 550
h.celsius = 34.0

In [ ]:
cell = Cell()

In [ ]:
# setup recording
n_rec = 1
t = h.Vector()
t.record(h._ref_t)
rec_vec = []
for i in range(n_rec):
    rec_vec.append( h.Vector() )
    rec_vec[-1].record(cell.soma[i](0.5)._ref_v)

### 5. Place some synapses at random dendritic locations

In [ ]:
synlist = []
NSYN = 10

In [ ]:
for i in range(NSYN):
    target = np.random.randint(len(cell.dend))
    synlist.append( h.ProbAMPANMDA_EMS(0.5, sec=cell.dend[target]) )

In [ ]:
stimlist = []
NSTIM = NSYN

### 6. Define how to stimulate those synapses

In [ ]:
for i in range(NSTIM):
    stim = h.NetStim()
    stim.start = np.random.normal(20.0,2.0)
    stim.noise = 1.0
    stim.interval = 100.0
    stim.number = 1e+9
    stimlist.append(stim)

In [ ]:
nclist = []
w = 5.0

In [ ]:
for syn, stim in zip(synlist[:-1], stimlist[:-1]):
    ncstim = h.NetCon(stim, syn)
    ncstim.delay = 1.
    ncstim.weight[0] = w
    nclist.append(ncstim)

### 7. Tell NEURON we want to record from every synapse

In [ ]:
t_vec = h.Vector()
id_vec = h.Vector()
for i in range(len(nclist)):
    nclist[i].record(t_vec, id_vec, i)

### 8. Initialize simulation state

In [ ]:
h.v_init = -88.734330774000796
h.init()
h.finitialize()

### 9. Simulate!

In [ ]:
h.tstop = 100
h.batch_run(100., 1./h.steps_per_ms)

In [ ]:
from matplotlib import gridspec
fig = plt.figure(figsize=(12, 6)) 
gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
ax0 = plt.subplot(gs[0])
for i in range(n_rec):
    ax0.plot(t,rec_vec[i],'-')
ax1 = plt.subplot(gs[1])
ax1.plot(t_vec.to_python(), id_vec.to_python(), '*')

### 10. Change some simulation parameters

Here we increase the frequency of stimulation of synapses

In [ ]:
for i in range(NSTIM):
    stimlist[i].interval = 1.0

In [ ]:
h.v_init = -88.734330774000796
h.init()
h.finitialize()

In [ ]:
h.tstop = 100
h.batch_run(100., 1./h.steps_per_ms)

In [ ]:
fig = plt.figure(figsize=(12, 6)) 
gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
ax0 = plt.subplot(gs[0])
for i in range(n_rec):
    ax0.plot(t,rec_vec[i],'-')
ax1 = plt.subplot(gs[1])
ax1.plot(t_vec.to_python(), id_vec.to_python(), '*')